In [1]:
import pandas as pd
import numpy as np
import imblearn
import scipy.stats as stats
from datetime import datetime 
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
import scipy.stats as stats

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score

from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline

In [53]:
df = pd.read_csv("merged-normalized.csv")
df = df.drop(["Date"],axis=1)
df["Time"]=pd.to_datetime(df["Time"], format='%H:%M:%S').dt.hour
#df["Time"] = stats.zscore(df["Time"])
df.head()

,Time,Race,Gender,Body_Size,With_Kids,Kids_Category,Basket_Size,Basket_colour,Attire,Shirt_Colour,...,buyDrinks,TotalSpent_RM,latitude,longitude,Num_of_Baskets,tempmax,tempmin,humidity,description,icon
0,20,malay,male,moderate,yes,young,big,red,casual,blue,...,-0.306655,-1.175289,-1.974354,0.219731,-1.24844,-0.804993,0.954235,0.51219,Cloudy skies throughout the day.,cloudy
1,20,malay,male,thin,no,no_kids,big,purple,casual,white,...,0.873818,1.620044,-1.844641,0.072282,1.25697,-0.804993,0.954235,0.51219,Cloudy skies throughout the day.,cloudy
2,20,malay,female,moderate,no,no_kids,big,blue,casual,red,...,-0.306655,-0.243511,-1.678802,1.008022,-1.24844,-0.804993,0.954235,0.51219,Cloudy skies throughout the day.,cloudy
3,21,indian,male,thin,no,no_kids,big,black,casual,black,...,-0.306655,0.455322,-1.495611,0.011978,-1.24844,-0.804993,0.954235,0.51219,Cloudy skies throughout the day.,cloudy
4,21,indian,male,fat,no,no_kids,big,blue,casual,blue,...,-0.896892,-0.243511,-2.086209,0.252582,1.25697,-0.804993,0.954235,0.51219,Cloudy skies throughout the day.,cloudy


In [80]:
#Label encode
label_encoder = preprocessing.LabelEncoder()
for col in df:
    if df[col].dtype == "object":
        df[col]=label_encoder.fit_transform(df[col])
        
df.rename(columns = {'icon':'weather'}, inplace = True)
#df.drop(["weather"],axis=1)
df.head()

,Time,Race,Gender,Body_Size,With_Kids,Kids_Category,Basket_Size,Basket_colour,Attire,Shirt_Colour,...,buyDrinks,TotalSpent_RM,latitude,longitude,Num_of_Baskets,tempmax,tempmin,humidity,description,weather
0,1.186818,1.279978,1.009041,0.003386,1.110019,1.512059,-0.849654,0.876722,-1.069717,-1.086611,...,-0.306655,-1.175289,-1.974354,0.219731,-1.24844,-0.729131,1.755352,0.320483,-1.616516,-4.855973
1,1.186818,1.279978,1.009041,1.234541,-0.900885,-0.542369,-0.849654,0.570658,-1.069717,1.258698,...,0.873818,1.620044,-1.844641,0.072282,1.25697,-0.729131,1.755352,0.320483,-1.616516,-4.855973
2,1.186818,1.279978,-0.991040,0.003386,-0.900885,-0.542369,-0.849654,-1.265730,-1.069717,0.965535,...,-0.306655,-0.243511,-1.678802,1.008022,-1.24844,-0.729131,1.755352,0.320483,-1.616516,-4.855973
3,1.330305,0.401626,1.009041,1.234541,-0.900885,-0.542369,-0.849654,-1.571795,-1.069717,-1.379775,...,-0.306655,0.455322,-1.495611,0.011978,-1.24844,-0.729131,1.755352,0.320483,-1.616516,-4.855973
4,1.330305,0.401626,1.009041,-1.227769,-0.900885,-0.542369,-0.849654,-1.265730,-1.069717,-1.086611,...,-0.896892,-0.243511,-2.086209,0.252582,1.25697,-0.729131,1.755352,0.320483,-1.616516,-4.855973


In [81]:
df =stats.zscore(df)
df.head()

,Time,Race,Gender,Body_Size,With_Kids,Kids_Category,Basket_Size,Basket_colour,Attire,Shirt_Colour,...,buyDrinks,TotalSpent_RM,latitude,longitude,Num_of_Baskets,tempmax,tempmin,humidity,description,weather
0,1.186818,1.279978,1.009041,0.003386,1.110019,1.512059,-0.849654,0.876722,-1.069717,-1.086611,...,-0.306655,-1.175289,-1.974354,0.219731,-1.24844,-0.729131,1.755352,0.320483,-1.616516,-4.855973
1,1.186818,1.279978,1.009041,1.234541,-0.900885,-0.542369,-0.849654,0.570658,-1.069717,1.258698,...,0.873818,1.620044,-1.844641,0.072282,1.25697,-0.729131,1.755352,0.320483,-1.616516,-4.855973
2,1.186818,1.279978,-0.991040,0.003386,-0.900885,-0.542369,-0.849654,-1.265730,-1.069717,0.965535,...,-0.306655,-0.243511,-1.678802,1.008022,-1.24844,-0.729131,1.755352,0.320483,-1.616516,-4.855973
3,1.330305,0.401626,1.009041,1.234541,-0.900885,-0.542369,-0.849654,-1.571795,-1.069717,-1.379775,...,-0.306655,0.455322,-1.495611,0.011978,-1.24844,-0.729131,1.755352,0.320483,-1.616516,-4.855973
4,1.330305,0.401626,1.009041,-1.227769,-0.900885,-0.542369,-0.849654,-1.265730,-1.069717,-1.086611,...,-0.896892,-0.243511,-2.086209,0.252582,1.25697,-0.729131,1.755352,0.320483,-1.616516,-4.855973


In [82]:
X = df.drop(["humidity"],axis=1)
y = df["humidity"]
colnames = X.columns

In [83]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [84]:
lr = LinearRegression()
rfe = RFECV(lr, min_features_to_select=10, cv=3, scoring="r2")

In [85]:
rfe.fit(X,y)

RFECV(cv=3, estimator=LinearRegression(), min_features_to_select=10,
      scoring='r2')

In [86]:
def ranking(ranks, names, order=1):
    minmax = MinMaxScaler() # everything will be between 0 and 1
    ranks = minmax.fit_transform(order*np.array([ranks]).T).T[0]
    ranks = map(lambda x: round(x,2), ranks)
    return dict(zip(names, ranks))

In [87]:
rfe_score = ranking(list(map(float, rfe.ranking_)), colnames, order=-1)
rfe_score = pd.DataFrame(list(rfe_score.items()), columns=['Features', 'Score'])
rfe_score = rfe_score.sort_values("Score", ascending = False)
rfe_score

,Features,Score
14,Spectacles,1.00
10,shirt_type,1.00
26,description,1.00
25,tempmin,1.00
24,tempmax,1.00
22,longitude,1.00
21,latitude,1.00
1,Race,1.00
13,Wash_Item,1.00
27,weather,1.00


In [88]:
from sklearn.model_selection import ShuffleSplit

optimal_X = X.iloc[:, rfe.support_]
optimal_X

,Race,Body_Size,shirt_type,Wash_Item,Spectacles,latitude,longitude,tempmax,tempmin,description,weather
0,1.279978,0.003386,0.900885,0.870327,-0.882365,-1.974354,0.219731,-0.729131,1.755352,-1.616516,-4.855973
1,1.279978,1.234541,0.900885,0.870327,-0.882365,-1.844641,0.072282,-0.729131,1.755352,-1.616516,-4.855973
2,1.279978,0.003386,0.900885,0.870327,-0.882365,-1.678802,1.008022,-0.729131,1.755352,-1.616516,-4.855973
3,0.401626,1.234541,0.900885,0.870327,-0.882365,-1.495611,0.011978,-0.729131,1.755352,-1.616516,-4.855973
4,0.401626,-1.227769,-1.110019,0.870327,1.133318,-2.086209,0.252582,-0.729131,1.755352,-1.616516,-4.855973
...,...,...,...,...,...,...,...,...,...,...,...
3995,-1.355078,1.234541,0.900885,-1.148993,1.133318,0.507268,0.156118,-1.404827,0.284508,-0.098482,0.512215
3996,0.401626,0.003386,-1.110019,0.870327,-0.882365,0.516629,0.368160,-1.404827,0.284508,-0.098482,0.512215
3997,0.401626,0.003386,-1.110019,0.870327,1.133318,0.544276,0.217933,-1.404827,0.284508,-0.098482,0.512215
3998,-1.355078,0.003386,0.900885,-1.148993,-0.882365,0.912267,0.384060,-1.404827,0.284508,-0.098482,0.512215


In [89]:
# evaluate model
cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
n_scores = cross_val_score(lasso,optimal_X, y, cv=cv, scoring='r2')
# report performance
print('R2: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

R2: 0.580 (0.014)


In [104]:
lasso = Lasso(alpha=0.1)
rfe2 = RFECV(lasso, cv=3, min_features_to_select=10, scoring="r2")

In [105]:
rfe2.fit(X,y)

RFECV(cv=3, estimator=Lasso(alpha=0.1), min_features_to_select=10, scoring='r2')

In [106]:
rfe_score = ranking(list(map(float, rfe2.ranking_)), colnames, order=-1)
rfe_score = pd.DataFrame(list(rfe_score.items()), columns=['Features', 'Score'])
rfe_score = rfe_score.sort_values("Score", ascending = False)
rfe_score

,Features,Score
14,Spectacles,1.00
15,Age_Range,1.00
26,description,1.00
25,tempmin,1.00
24,tempmax,1.00
23,Num_of_Baskets,1.00
22,longitude,1.00
21,latitude,1.00
20,TotalSpent_RM,1.00
19,buyDrinks,1.00


In [107]:
optimal_X = X.iloc[:, rfe2.support_]
optimal_X

,Shirt_Colour,shirt_type,Pants_Colour,pants_type,Wash_Item,Spectacles,Age_Range,Washer_No,Dryer_No,TimeSpent_minutes,buyDrinks,TotalSpent_RM,latitude,longitude,Num_of_Baskets,tempmax,tempmin,description,weather
0,-1.086611,0.900885,-1.182103,1.089490,0.870327,-0.882365,-0.979143,-1.329343,1.383455,-1.095669,-0.306655,-1.175289,-1.974354,0.219731,-1.248440,-0.729131,1.755352,-1.616516,-4.855973
1,1.258698,0.900885,-0.635434,-0.917861,0.870327,-0.882365,-0.635820,1.333782,0.483647,-1.165587,0.873818,1.620044,-1.844641,0.072282,1.256970,-0.729131,1.755352,-1.616516,-4.855973
2,0.965535,0.900885,-1.182103,-0.917861,0.870327,-0.882365,-0.807481,-0.441635,1.383455,-1.235504,-0.306655,-0.243511,-1.678802,1.008022,-1.248440,-0.729131,1.755352,-1.616516,-4.855973
3,-1.379775,0.900885,1.824575,1.089490,0.870327,-0.882365,0.994965,0.446073,0.483647,-1.585093,-0.306655,0.455322,-1.495611,0.011978,-1.248440,-0.729131,1.755352,-1.616516,-4.855973
4,-1.086611,-1.110019,1.551241,-0.917861,0.870327,1.133318,-0.464158,-1.329343,0.483647,-1.445258,-0.896892,-0.243511,-2.086209,0.252582,1.256970,-0.729131,1.755352,-1.616516,-4.855973
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,-1.379775,0.900885,-0.908768,1.089490,-1.148993,1.133318,-0.120835,-0.441635,-0.416161,0.722193,1.464055,1.620044,0.507268,0.156118,0.004265,-1.404827,0.284508,-0.098482,0.512215
3996,0.379207,-1.110019,1.824575,-0.917861,0.870327,-0.882365,1.338288,-0.441635,0.483647,-1.515175,-1.487129,-0.942345,0.516629,0.368160,0.004265,-1.404827,0.284508,-0.098482,0.512215
3997,-1.379775,-1.110019,-1.182103,1.089490,0.870327,1.133318,1.595781,-0.441635,0.483647,1.071782,-0.306655,0.455322,0.544276,0.217933,-1.248440,-1.404827,0.284508,-0.098482,0.512215
3998,0.965535,0.900885,1.824575,1.089490,-1.148993,-0.882365,-1.150805,0.446073,1.383455,-0.186738,0.873818,0.222378,0.912267,0.384060,1.256970,-1.404827,0.284508,-0.098482,0.512215


In [108]:
# evaluate model
cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
n_scores = cross_val_score(lasso,optimal_X, y, cv=cv, scoring='r2')
# report performance
print('R2: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

R2: 0.544 (0.008)


In [114]:
ridge = Ridge(alpha=0.9)
rfe3 = RFECV(ridge, cv=3, min_features_to_select=10,scoring="r2")

In [115]:
rfe3.fit(X,y)

RFECV(cv=3, estimator=Ridge(alpha=0.9), min_features_to_select=10, scoring='r2')

In [116]:
rfe_score = ranking(list(map(float, rfe3.ranking_)), colnames, order=-1)
rfe_score = pd.DataFrame(list(rfe_score.items()), columns=['Features', 'Score'])
rfe_score = rfe_score.sort_values("Score", ascending = False)
rfe_score

,Features,Score
14,Spectacles,1.00
10,shirt_type,1.00
26,description,1.00
25,tempmin,1.00
24,tempmax,1.00
22,longitude,1.00
21,latitude,1.00
1,Race,1.00
13,Wash_Item,1.00
27,weather,1.00


In [117]:
optimal_X = X.iloc[:, rfe3.support_]
optimal_X

,Race,Body_Size,shirt_type,Wash_Item,Spectacles,latitude,longitude,tempmax,tempmin,description,weather
0,1.279978,0.003386,0.900885,0.870327,-0.882365,-1.974354,0.219731,-0.729131,1.755352,-1.616516,-4.855973
1,1.279978,1.234541,0.900885,0.870327,-0.882365,-1.844641,0.072282,-0.729131,1.755352,-1.616516,-4.855973
2,1.279978,0.003386,0.900885,0.870327,-0.882365,-1.678802,1.008022,-0.729131,1.755352,-1.616516,-4.855973
3,0.401626,1.234541,0.900885,0.870327,-0.882365,-1.495611,0.011978,-0.729131,1.755352,-1.616516,-4.855973
4,0.401626,-1.227769,-1.110019,0.870327,1.133318,-2.086209,0.252582,-0.729131,1.755352,-1.616516,-4.855973
...,...,...,...,...,...,...,...,...,...,...,...
3995,-1.355078,1.234541,0.900885,-1.148993,1.133318,0.507268,0.156118,-1.404827,0.284508,-0.098482,0.512215
3996,0.401626,0.003386,-1.110019,0.870327,-0.882365,0.516629,0.368160,-1.404827,0.284508,-0.098482,0.512215
3997,0.401626,0.003386,-1.110019,0.870327,1.133318,0.544276,0.217933,-1.404827,0.284508,-0.098482,0.512215
3998,-1.355078,0.003386,0.900885,-1.148993,-0.882365,0.912267,0.384060,-1.404827,0.284508,-0.098482,0.512215


In [118]:
# evaluate model
cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
n_scores = cross_val_score(ridge,optimal_X, y, cv=cv, scoring='r2')
# report performance
print('R2: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

R2: 0.580 (0.014)


In [119]:
from sklearn.preprocessing import PolynomialFeatures

In [120]:
poly = PolynomialFeatures(degree=2, include_bias=False)

In [121]:
poly_features = poly.fit_transform(X)

In [122]:
poly_model = LinearRegression()
rfe_poly = RFECV(poly_model, min_features_to_select=10, cv=3)
rfe_poly.fit(poly_features,y)

RFECV(cv=3, estimator=LinearRegression(), min_features_to_select=10)

In [123]:
rfe_score = ranking(list(map(float, rfe_poly.ranking_)), colnames, order=-1)
rfe_score = pd.DataFrame(list(rfe_score.items()), columns=['Features', 'Score'])
rfe_score = rfe_score.sort_values("Score", ascending = False)
rfe_score

,Features,Score
14,Spectacles,1.00
10,shirt_type,1.00
26,description,1.00
25,tempmin,1.00
24,tempmax,1.00
22,longitude,1.00
21,latitude,1.00
18,TimeSpent_minutes,1.00
13,Wash_Item,1.00
27,weather,1.00


In [ ]:
pipeline = Pipeline(steps=[('s',rfe_poly),('m',poly_model)])
# evaluate model
cv = ShuffleSplit(n_splits=5,train_size=0.6, test_size=0.3, random_state=0)
n_scores = cross_val_score(pipeline, poly_features, y, cv=cv, scoring='r2')
# report performance
print('R2: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))